# Linear Panel Estimations



In [71]:
import pandas as pd 
import numpy as np
import seaborn as sns
from numpy import linalg as la
from scipy.stats import chi2
from tabulate import tabulate
import LinearModelsProject1 as lm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

## Reading data from the csv file
data = pd.read_csv("firms.csv")

N_list = data.firmid.unique()
T_list = data.year.unique()

N = data.firmid.unique().size
T = data.year.unique().size

y = data.ldsa.values.reshape((N*T,1))
l = data.lemp.values.reshape((N*T,1))
k = data.lcap.values.reshape((N*T,1))

constant = np.ones((y.shape[0], 1))
X = np.hstack([constant, l, k])





In [73]:
# Usi the POLS method to estimate parameters

label_x = ["c", "Log Employment", "Log Adjusted Capital"]
label_y = "Log Deflated Sales"


ols_result = lm.estimate(y, X)

lm.print_table(
    (label_y, label_x), ols_result, title="Pooled OLS", floatfmt='.4f'
)


Pooled OLS
Dependent variable: Log Deflated Sales

                        Beta      Se    t-values
--------------------  ------  ------  ----------
c                     0.0000  0.0050      0.0000
Log Employment        0.6748  0.0102     66.4625
Log Adjusted Capital  0.3100  0.0091     33.9237
R² = 0.914
σ² = 0.131


In [74]:
# Estimate using the FE method

def demeaning_matrix(T):
    Q_T = np.eye(T) - np.tile(1/T, (T, T))
    return Q_T

Q_T = demeaning_matrix(T)


y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, X)
x_demean = x_demean[:, 1:]
label_x_fe = label_x[1:]


fe_result = lm.estimate(
    y_demean, x_demean, transform='fe', T=T, robust_se=True
)


lm.print_table(
    (label_y,label_x_fe), 
    fe_result, title='FE regression', floatfmt='.4'
)



FE regression
Dependent variable: Log Deflated Sales

                        Beta       Se    t-values
--------------------  ------  -------  ----------
Log Employment        0.6942  0.04165      16.67
Log Adjusted Capital  0.1546  0.02995       5.163
R² = 0.477
σ² = 0.018


In [75]:
# Estimate using the FD method

def fd_matrix(T):
    D_T = np.eye(T) - np.eye(T, k=-1)
    D_T = D_T[1:]
    return D_T

D_T = fd_matrix(T)

y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, X[:,1:])
fd_result = lm.estimate(y_diff, x_diff, transform='fd', T=T, robust_se=True)

lm.print_table(
    (label_y, label_x[1:]), 
    fd_result, title='FD regression', floatfmt='.4f'
)


FD regression
Dependent variable: Log Deflated Sales

                        Beta      Se    t-values
--------------------  ------  ------  ----------
Log Employment        0.5487  0.0284     19.3056
Log Adjusted Capital  0.0630  0.0229      2.7460
R² = 0.165
σ² = 0.014
